In [30]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [45]:
# import numpy as np
import pandas as pd
import numpy as np

import yfinance as yf
import bokeh

from threading import Thread
import time
from functools import partial

from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, HoverTool,CrosshairTool, Div,Paragraph, Spinner, Span, MultiChoice, Dropdown
from bokeh.plotting import ColumnDataSource, figure, output_file, show, curdoc
from bokeh.models import NumeralTickFormatter, Panel, Tabs, PreText, DatePicker, Select, Quadratic, TextInput
import bokeh.plotting as bk
from bokeh.palettes import d3

from datetime import datetime, date, timedelta

from bokeh.io import show, save, output_notebook, reset_output
from bokeh.models import CustomJS, RangeSlider, DataRange1d, Div

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

from tornado import gen
import locale

#[X] Stock Splits
#[X] Organize by tab
#[X] Create different source for each tab
#[X] Last day always equals current value 
#[] Function string to dates
# [] consistent number scheme for tabs and variables 
#[X] index fund includes only trading days
# [X] X axis not updating
# [X] Div sign 
# [] source.update(callback) so other tabs don't change 
# [] Minimum dates based on tickers
    
output_notebook()
locale.setlocale(locale.LC_ALL, '')
def modify_doc(doc):
        
    def yf_fund(ticker, start_date, end_date, principal):
        df_yf_fund = pd.DataFrame()
        yf_fund_ticker = yf.Ticker(ticker)

        end_date2 = end_date + timedelta(1)
        end_date2 = str(end_date2)
        start_date = str(start_date)
        
        df_yf_fund = yf_fund_ticker.history(start=start_date, end=end_date2)
        yf_fund_cost_basis = df_yf_fund.iloc[0, 3]
        no_shares = principal/yf_fund_cost_basis
        df_yf_fund['Position'] = df_yf_fund['Close'] * no_shares
        for i in df_yf_fund[df_yf_fund['Stock Splits'] > 0].index.values:
            df_yf_fund.at[i, 'Position'] *= df_yf_fund.at[i, 'Stock Splits']

        return df_yf_fund, yf_fund_cost_basis
    
    def managed_fund(principal, current_value, df_yf_fund):
        start_date = pd.to_datetime(df_yf_fund.index[0]) 
        end_date = pd.to_datetime(df_yf_fund.index[-1]) 
        period = (end_date - start_date).days
        period_years = period/365.25
        rate = ((current_value/principal)**(1/period_years)) - 1
        
        df_managed_fund = pd.DataFrame()
        df_managed_fund['Date'] = [(start_date + timedelta(i)) for i in range(period + 1)]
        df_managed_fund['Date'] = pd.to_datetime(df_managed_fund['Date'])
        df_managed_fund['Position'] = [principal * (1 + rate) ** (i/365.25) for i in range(period + 1)]
        df_managed_fund = df_managed_fund[df_managed_fund['Date'].isin(df_yf_fund.index.values)]
        df_managed_fund = df_managed_fund.set_index('Date')
        df_managed_fund.columns = [f'Managed {i}' for i in df_managed_fund.columns]
        
        return df_managed_fund, rate
        
    #def create_source(df_fund1, df_fund2, col1, col2, ticker1, ticker2):
    def create_source(df_fund1, df_fund2, ticker1, ticker2):
        df_fund1.index = pd.to_datetime(df_fund1.index)
        df_fund2.index = pd.to_datetime(df_fund2.index)
        
        col1 = next(i for i in df_fund1.columns if 'Position' in i)
        col2 = next(i for i in df_fund2.columns if 'Position' in i)
                
        df_source = df_fund1.join(df_fund2)
        df_source['Difference'] = df_fund1[col1] - df_fund2[col2]
        df_source['legend1'] = np.zeros_like(df_source['Difference'])
        df_source['legend1'] = ticker1
        df_source['legend2'] = np.zeros_like(df_source['Difference'])
        df_source['legend2'] = ticker2

        return df_source
    
    def make_plot(df_source, source, title):
        position_col = [i for i in df_source.columns if 'Position' in i]
        line1 = position_col[0]
        line2 = position_col[1]
              
        plot = figure(width_policy = 'fit', height_policy = 'fit', x_axis_type='datetime', title = title)
        plot.line('Date', line1, source = source, legend_field = 'legend1', color = '#1f77b4', line_width = 3)
        plot.line('Date', line2, source = source, legend_field = 'legend2', color = d3['Category10'][10][1], line_width = 3)
        plot.add_tools(CrosshairTool())
        plot.legend.location = 'top_left'
        plot.legend.click_policy = 'hide'
        plot.xaxis.axis_label = 'Date'
        plot.yaxis.axis_label = 'USD ($)'
        
        return plot        

    def div_text(df_source, ticker1, ticker2, cost_basis, investment_type):
        position_col = next(i for i in df_source.columns if 'Position' in i)
        current_value = df_source[position_col][-1]
        principal = df_source[position_col][0]
        growth = (current_value - principal)/principal 
        verb = 'appreciated' if growth > 0 else 'depreciated'
        difference = df_source['Difference'][-1]
        pos_sign = '+' if float(difference) > 0 else ''
        difference = locale.currency(difference, grouping = True)
        cost_basis = locale.currency(cost_basis, grouping = True)
        text =(
              f'Your {investment_type} {verb} {growth: .0%}.'
              f'<br>If you invested in {ticker1}, you would have {pos_sign}{difference}.'
              f'<br>Cost basis for {ticker2}: {cost_basis}/share'
              )
        
        return text
    
    # Callbacks all 
    def update(attrname, old, new):
        start_date = pd.to_datetime(start_date_picker.value).date()
        end_date = pd.to_datetime(end_date_picker.value).date()
        principal = principal_spinner.value
        current_value = current_value_spinner.value
        ticker = ticker_select.value
        
        df_index_fund, index_cost_basis = yf_fund(ticker_symbols[ticker], start_date, end_date, principal)
        df_index_fund.columns = [f'Index {i}' for i in df_index_fund.columns]
        df_stock, stock_cost_basis = yf_fund(stock_input.value, start_date, end_date, principal)
        df_stock.columns = [f'Stock {i}' for i in df_stock.columns]
        
        df_managed_fund, rate = managed_fund(principal, current_value, df_index_fund)
        df_source1 = create_source(df_index_fund, df_managed_fund, ticker, 'Managed Position')
                    
        new_source1 = ColumnDataSource(df_source1)
        source1.data.update(new_source1.data)
        div.text = div_text(df_source1, ticker, ticker, index_cost_basis, 'mutual fund')   
        
        # Tab 2
        df_stock, stock_cost_basis = yf_fund(stock_input.value, start_date, end_date, principal)
        df_stock.columns = [f'Stock {i}' for i in df_stock.columns]    
        df_source2 = create_source(df_index_fund, df_stock, ticker, stock_input.value)
        
        new_source2 = ColumnDataSource(df_source2)
        source2.data.update(new_source2.data)
        
        current_value_stock = source2.data['Stock Position'][-1]
        div2.text = div_text(df_source2, ticker, stock_input.value,
                              stock_cost_basis, f'{stock_input.value} investment')
        
        # Tab 3
        df_stock2, stock2_cost_basis = yf_fund(stock2_input.value, start_date, end_date, principal)
        df_stock2.columns = [f'Stock 2 {i}' for i in df_stock2.columns]  
        df_source3 = create_source(df_stock2, df_stock, stock_input.value, stock2_input.value)
        
        new_source3 = ColumnDataSource(df_source3)
        source3.data.update(new_source3.data)        
    
        current_value_stock2 = source3.data['Stock Position'][-1]
        div3.text = div_text(df_source3, stock2_input.value, stock_input.value,
                          stock_cost_basis, f'{stock_input.value} invesment')
    
    ticker_symbols = {
        'DJI': '^DJI',
        'S&P 500': '^GSPC'
    }
    
    #WIDGETS
    
    principal = 1000.0
    current_value = 3000.0
    ticker = 'S&P 500'
    start_date = date(2021, 5, 3)
    end_date = date(2021, 5, 7)
    
    start_date_picker = DatePicker(title = 'Start Date', value = start_date)#, min_date="2019-08-01", max_date="2019-10-30")
    end_date_picker = DatePicker(title = 'End Date', value = end_date) 
    principal_spinner = Spinner(value=principal, step=1, title='Principal')
    current_value_spinner = Spinner(value=current_value, step=1, title='Current Value')
    ticker_select = Select(title='Index Fund', value = 'S&P 500', options = ['DJI', 'S&P 500'])
    stock_input = TextInput(value="AMZN", title="Stock 1 Ticker Symbol")
    stock2_input = TextInput(value="GOOG", title="Stock 2 Ticker Symbol")

    # Tab 1
    #Get data
        
    df_index_fund, index_cost_basis = yf_fund(ticker_symbols[ticker], start_date, end_date, principal)
    df_index_fund.columns = [f'Index {i}' for i in df_index_fund.columns]
    
    df_managed_fund, rate = managed_fund(principal, current_value, df_index_fund)

    df_source1 = create_source(df_index_fund, df_managed_fund, ticker, 'Managed Position')
    source1 = ColumnDataSource(df_source1)

    #Set-up Plots
    
    TOOLTIPS = [
                ('Date','@Date{%F}'),
                ('Index Fund', '@{Index Position}{$0,0}'),        
                ('Managed Fund',"@{Managed Position}{$0,0}"),
                ("Difference","@Difference{$0,0}"),                
                ]    
    
    plot = make_plot(df_source1, source1, 'Managed Fund vs. Index Fund')
    plot.add_tools(HoverTool(tooltips = TOOLTIPS, formatters={'@Date': 'datetime'}))

    div = Div(text = div_text(df_source1, ticker, ticker, index_cost_basis, 'mutual fund'), 
              sizing_mode = 'stretch_width', height=100)    
        
    # Layout
    
    inputs = column(principal_spinner, current_value_spinner, ticker_select, start_date_picker, end_date_picker, div)
    tab_managed = Panel(child = row(plot, inputs), title = 'Managed Fund vs Index Fund')
  
    # Tab2
    
    # Data
    
    df_stock, stock_cost_basis = yf_fund(stock_input.value, start_date, end_date, principal)
    df_stock.columns = [f'Stock {i}' for i in df_stock.columns]    
    
    df_source2 = create_source(df_index_fund, df_stock, ticker, stock_input.value)
    source2 = ColumnDataSource(df_source2)    
    
    # Plots
    TOOLTIPS2 = [
                ('Date','@Date{%F}'),      
                ('Index Fund', "@{Index Position}{$0,0}"),
                ('Stock','@{Stock Position}{$0,0}'),
                ("Difference","@{Difference}{$0,0}"),                
                ]
            
    plot2 = make_plot(df_source2, source2, 'Stock vs. Index Fund')
    plot2.add_tools(HoverTool(tooltips = TOOLTIPS2, formatters={'@Date': 'datetime'}))
    
    current_value_stock = source2.data['Stock Position'][-1]
    div2 = Div(text = div_text(df_source2, ticker, stock_input.value,
                              stock_cost_basis, f'{stock_input.value} investment'), 
               sizing_mode = 'stretch_width', height=100)
    # Layout
    
    inputs_stock = column(principal_spinner, stock_input, ticker_select, start_date_picker, end_date_picker, div2)
    tab_stock = Panel(child = row(plot2, inputs_stock), title = 'Stock vs Index Fund')
    
    # Tab3
    
    # Data
    
    df_stock2, stock2_cost_basis = yf_fund(stock2_input.value, start_date, end_date, principal)
    df_stock2.columns = [f'Stock 2 {i}' for i in df_stock2.columns]  
    df_source3 = create_source(df_stock2, df_stock, stock_input.value, stock2_input.value)
    source3 = ColumnDataSource(df_source3) 
    
    # Plot
    
    TOOLTIPS3 = [
                ('Date','@Date{%F}'),
                ('Stock 1',"@{Stock Position}{$0,0}"),
                ('Stock 2',"@{Stock 2 Position}{$0,0}"),
                ("Difference","@{Difference}{$0,0}"),                
                ]
              
    plot3 = make_plot(df_source3, source3, 'Stock vs. Stock')
    plot3.add_tools(HoverTool(tooltips = TOOLTIPS3, formatters={'@Date': 'datetime'}))
    
    current_value_stock2 = source3.data['Stock Position'][-1]
    div3 = Div(text = div_text(df_source3, stock2_input.value, stock_input.value, 
                          stock_cost_basis, f'{stock_input.value} investment'), 
               sizing_mode = 'stretch_width', height=100)
    
    # Layout
    
    inputs_stock2 = column(principal_spinner, stock_input, stock2_input, start_date_picker, end_date_picker, div3)        
    tab_stock2 = Panel(child = row(plot3, inputs_stock2), title = 'Stock vs Stock')
        
    start_date_picker.on_change('value', update)
    end_date_picker.on_change('value', update)
    principal_spinner.on_change('value', update)
    current_value_spinner.on_change('value', update)
    ticker_select.on_change('value', update)    
    stock_input.on_change('value', update)
    stock2_input.on_change('value', update)
    
    #Set-up layout 

    layout = Tabs(tabs=[tab_managed, tab_stock, tab_stock2])
    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app) #, notebook_url="localhost:8888")

Loading BokehJS ...